В этом блокноте представлено построение и сохранение модели BERTopic на непредобработанных данных. На выходе имеем датасет link|num_of_topic - сохранение его. (почему -то так оно работает лучше, чем с предобработанными...однако, слова фильм, этот и т.п. всё же мешаются)

In [ ]:
!pip install transformers
!pip install bertopic

In [ ]:
from bertopic import BERTopic

In [ ]:
import numpy as np
import pandas as pd
import torch

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#загрузка полного датасета
mydf=pd.read_csv('/content/drive/MyDrive/model/final_df100.csv', index_col=0)
df = mydf[['id', 'review']].drop_duplicates(subset=['review'],ignore_index= True) # удаляю повторы комментариев

In [ ]:
df = df.sort_values(by = ['id'], ignore_index = True)
df = df.groupby('id').head(5) #датафрейм, в котором нет дубликатов и комментариев к одному фильму не более 5 !!!!!

# обучение и сохранение модели
docs_ = df["review"].to_list()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#вручную задаю количество тем 200 !!!!!
topic_model = BERTopic(language="russian", nr_topics = 200)

topics, _ = topic_model.fit_transform(docs_)

topic_model.save('/content/drive/MyDrive/model/mm3')

2024-04-06 16:54:54,474 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
# загрузка ранее рассчитанной модели BERTopic (Важно, делать это нужно на GPU) модель mm - на непредобработанных данных!!! просто эксперимент
topic_model = BERTopic.load('/content/drive/MyDrive/model/mm3')

In [ ]:
a = [0]*len(df['review']) # определение самой подходящей темы для каждого фильма из датасета
for i in range(len(df['review'])):
  a[i] = topic_model.find_topics(df['review'][i], top_n = 1)[0][0]

link_list = [None]*len(df['id']) # список ссылок в таблицу
for i in range(len(df['id'])):
  link_list[i] = 'https://www.kinopoisk.ru/film/'+df['id'][i].__str__()+'/'

top_df = df.assign(link = link_list, topic = a)
top_df_3 = top_df[top_df.topic > 1]


In [ ]:
# СОХРАНЕНИЕ ТАБЛИЦЫ
top_df_3.to_csv('top_df_3.csv')
!cp top_df_3.csv "drive/My Drive/model"

In [ ]:
mymes = "обрести веру в любовь и настоящие искренние чувства"

def return_links(my_mes):
    my_topic = topic_model.find_topics(my_mes, top_n = 1)[0][0]
    return top_df_3[top_df_3.topic  == my_topic].link.tail(30) #вывод подходящих ссылок

return_links(mymes)